In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import svd

In [ ]:
apikey1 = '98WO4EK4CGOBTPQR'
apikey2 = '7HTSH49XTMKQSSUI'
premiumkey = 'D6W40P80TZPAE9QB'

In [ ]:
#This one gives adjusted 
def TimeSeriesIntraday_df(stock_name):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={stock_name}&extended_hours=false&interval=1min&outputsize=full&apikey={premiumkey}&datatype=csv'
    df = pd.read_csv(url)
    df['symbol'] = stock_name
    return df
    

In [ ]:
def get_intraday_prices_chg(portfolio):
    """
    Returns a DataFrame of aligned close prices for the given portfolio,
    and a percentage change array.
    """
    data = {}

    # Build dict of Series indexed by timestamp
    for s in portfolio:
        df = TimeSeriesIntraday_df(s)
        df = df.sort_values('timestamp')  # ensure ascending order
        prices = df.set_index('timestamp')['close']
        data[s] = prices

    # Align on common timestamps only
    df_combined = pd.concat(data.values(), axis=1, join='inner')
    df_combined.columns = data.keys()  # Rename columns to match symbols
    df_combined = df_combined.sort_index()  # Ensure datetime order ascending

    df_combined['mean'] = df_combined.mean(axis=1)
    price_arr = df_combined.to_numpy().T  # shape: (assets+1, time steps)

    # Percent change calculation (skip 'mean' row in price_arr)
    return_arr = (price_arr[:-1, 1:] - price_arr[:-1, :-1]) / price_arr[:-1, :-1] * 100

    return price_arr, return_arr


In [ ]:
portfolio_daily = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA', 'NDAQ']
portfolio = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA']

In [ ]:
intraday_data, intraday_chg_data = get_intraday_prices(portfolio_daily)